# Endpointtest SongsMS

In [1]:
import requests
import json

### Configurations

In [2]:
port_service_auth = "8080"
port_service_songs = "8080"
port_service_lyrics = "8080"


adress = "http://localhost"

url_auth = adress + ":" + port_service_auth + "/auth"
url_songs = adress + ":" + port_service_songs + "/songs"
url_songLists = adress + ":" + port_service_songs + "/songLists"
url_lyrics = adress + ":" + port_service_lyrics + "/lyrics"


# set login data for authentication
userIdM = "mmuster"
userIdS = "eschuler"
pw = "pass1234"

# get an Authorization Key
def auth(userId):
    return requests.request(
        "POST",
        url_auth,
        headers={'Content-Type': 'application/json'},
        data=json.dumps({"userId": userId, "password": pw})
    ).text


# set global authorization
auth_keyM = auth(userIdM)
auth_keyS = auth(userIdS)
assert auth_keyM != ""
assert auth_keyM != ""
print("Configuration done. AuthKey: " + auth_keyM + " & " + auth_keyS)


# set header with different authorization mode (wrong key, empty key, no key or working key as default)
def header(auth="default", authKey=auth_keyM):
    if auth == "wrong":
        headers = {
            'Authorization': authKey + "d",
            'Content-Type': 'application/json'
        }

    elif auth == "empty":
        headers = {
            'Authorization': "",
            'Content-Type': 'application/json'
        }

    elif auth == "nonexist":
        headers = {
            'Content-Type': 'application/json'
        }

    else:
        headers = {
            'Authorization': authKey,
            'Content-Type': 'application/json'
        }
    return headers


def checkSimpleRequests(testCases, requestMethod, url):
    testCases
    for i in range(len(testCases)):
        #id , headers, status_code, payload, response(expected), res-header(expected)
        id = testCases[i][0]
        testHeader = testCases[i][1]
        testStatus_code = testCases[i][2]
        testPayload = testCases[i][3]
        testResponseExpect = testCases[i][4]
        testHeaderExpect = testCases[i][5]
        print(str(i+1) + " Send request: " + requestMethod, url +
              id, str(testHeader)[:37], str(testPayload)[:40])
        response = requests.request(
            requestMethod, url+id, headers=testHeader, data=testPayload)
        print(str(response.headers))
        print(" -Recieved: " + str(response.status_code) +
              " Body: "+str(response.text))
        print(" -Expected: " + str(testStatus_code) +
              " Body: " + str(testResponseExpect))
        if not response.status_code in [201, 400, 404, 405]:
            assert(str(response.text) == str(testResponseExpect)
                   ), "Response Body Different"
        assert(testStatus_code == response.status_code), "Status Code Different"
        if(testHeaderExpect != ""):
            assert(str(response.headers).find(testHeaderExpect)
                   != 0), "Response Header not as aspected"
        if(response.status_code == 204 and requestMethod == "PUT"):
            print(str(requests.request("GET", url+id, headers=testHeader).text))

def clear(count=10):
    #clear all songLists (for both default users) and songs (without default entrys from data.sql/import.sql)
    for i in range(count):
        requests.request("DELETE", url_songLists + "/" + str(i+5), headers=header("default", auth_keyM))
        requests.request("DELETE", url_songLists + "/" + str(i+5), headers=header("default", auth_keyS))
        requests.request("DELETE", url_songs + "/" + str(i+3), headers=header("default",auth_keyM))
clear()

Configuration done. AuthKey: 29eEJR8VvxyjuTVm & l51JRjuixOvXRfWE


# Auth Test

### Setup Payloads / Test Cases

In [3]:
payload = []
payload.append(json.dumps({
  "userId": userIdM,
  "password": pw
}))

payload.append(json.dumps({
  "userId": userIdM,
  "password": "wrong"
}))

payload.append(json.dumps({
  "userId": "wrong",
  "password": pw
}))

payload.append(json.dumps({
  "userId": "wrong",
  "password": "wrong"
}))

payload.append(json.dumps({
  "userId": "wrong",
  "password": ""
}))

payload.append(json.dumps({
  "userId": "",
  "password": "wrong"
}))

payload.append(json.dumps({
  "userId": userIdM,
  "password": ""
}))

payload.append(json.dumps({
  "userId": "",
  "password": pw
}))

payload.append(json.dumps({
  "userId": "",
  "password": ""
}))

payload.append(json.dumps({
  "userId": userIdM
}))

payload.append(json.dumps({
  "userId": "wrong"
}))

payload.append(json.dumps({
  "userId": ""
}))

payload.append(json.dumps({
}))

## Auth Post Methods

### Starting requests and store

In [4]:
# succesfull test
authSuccess = requests.request("POST", url_auth, headers=header("nonexist"), data=payload[0])
print("Recieved Statuscode: " + str(authSuccess.status_code) + " Expected is 200")
assert(200 == authSuccess.status_code)

#tests with wrong authentication data should fail
auths=[]

for i in range(len(payload)-1):
  auths.append(["",header("nonexist"),401,payload[i+1],"",""])

checkSimpleRequests(auths, "POST", url_auth)
  

Recieved Statuscode: 200 Expected is 200
1 Send request: POST http://localhost:8080/auth {'Content-Type': 'application/json'} {"userId": "mmuster", "password": "wrong
{'Date': 'Thu, 30 Sep 2021 17:24:27 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
2 Send request: POST http://localhost:8080/auth {'Content-Type': 'application/json'} {"userId": "wrong", "password": "pass123
{'Date': 'Thu, 30 Sep 2021 17:24:27 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
3 Send request: POST http://localhost:8080/auth {'Content-Type': 'application/json'} {"userId": "wrong", "password": "wrong"}
{'Date': 'Thu, 30 Sep 2021 17:24:27 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
4 Send request: POST http://localhost:8080/auth {'Content-Type': 'application/json'} {"userId": "wrong", "password": ""}
{'Date': 'Thu, 30 Sep 2021 17:24:27 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
5 Send request: 

## Auth Get Methods

### Identify User with token 


In [5]:
auth_keyM = auth(userIdM)
auth_keyS = auth(userIdS)


auths = [
    ["",header("default",auth_keyM),200,None,userIdM,""],
    ["",header("default",auth_keyS),200,None,userIdS,""],
    ["",header("wrong"),401,None,"",""],
    ["",header("empty"),401,None,"",""],
    ["",header("nonexist"),400,None,"",""]
]

checkSimpleRequests(auths, "GET", url_auth)

1 Send request: GET http://localhost:8080/auth {'Authorization': 'zgFiqHXUyFhbStb0', None
{'Content-Type': 'text/plain;charset=UTF-8', 'Date': 'Thu, 30 Sep 2021 17:24:27 GMT', 'content-length': '7'}
 -Recieved: 200 Body: mmuster
 -Expected: 200 Body: mmuster
2 Send request: GET http://localhost:8080/auth {'Authorization': 'YBZbwmzyuozXsl6n', None
{'Content-Type': 'text/plain;charset=UTF-8', 'Date': 'Thu, 30 Sep 2021 17:24:28 GMT', 'content-length': '8'}
 -Recieved: 200 Body: eschuler
 -Expected: 200 Body: eschuler
3 Send request: GET http://localhost:8080/auth {'Authorization': '29eEJR8VvxyjuTVmd' None
{'Date': 'Thu, 30 Sep 2021 17:24:28 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
4 Send request: GET http://localhost:8080/auth {'Authorization': '', 'Content-Type': None
{'Date': 'Thu, 30 Sep 2021 17:24:28 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
5 Send request: GET http://localhost:8080/auth {'Content-Type': 'application/j

# Songs


### Setup

In [6]:
jsonSong = json.dumps({
  "title": "Come To Me",
  "artist": "AETNA",
  "label": "unknown",
  "released": 2020
}, separators=(',', ':'))

jsonSongWithOnlyTitle = json.dumps({
  "title": "Come To Me Only Title"
}, separators=(',', ':'))

jsonSongWithEmptyTitle = json.dumps({
  "title": "",
  "artist": "AETNA with Empty Title",
  "label": "unknown",
  "released": 2020
}, separators=(',', ':'))

jsonSongWithOutTitle = json.dumps({
  "artist": "AETNA Without Title",
  "label": "unknown",
  "released": 2020
}, separators=(',', ':'))


## Songs Get Methods

In [7]:
print(header())

jsonSongCompare1 = json.dumps({
    "id":1,
    "title":"We Built This City",
    "artist":"Starship",
    "label":"Grunt/RCA",
    "released":1985
    }, separators=(',', ':'))

jsonSongCompare2 = json.dumps({
    "id":2,
    "title":"Sussudio",
    "artist":"Phil Collins",
    "label":"Virgin",
    "released":1985
    }, separators=(',', ':'))

jsonSongCompare3 = json.dumps([{
    "id":1,
    "title":"We Built This City",
    "artist":"Starship",
    "label":"Grunt/RCA",
    "released":1985
    },
    {
    "id":2,
    "title":"Sussudio",
    "artist":"Phil Collins",
    "label":"Virgin",
    "released":1985
    }], separators=(',', ':'))

gets = [
    #id , headers, status_code, payload, response(expected), res-header(expected)
    ["/1",header("default",auth_keyM),200,None,jsonSongCompare1,""],
    ["/1",header("wrong"),401,None,"",""],
    ["/1",header("empty"),401,None,"",""],
    ["/1",header("nonexist"),400,None,"",""],
    ["/2",header("default",auth_keyM),200,None,jsonSongCompare2,""],
    ["/3",header("default",auth_keyM),404,None,"",""],
    ["/4",header("default",auth_keyM),404,None,"",""],
    ["",header("default",auth_keyM),200,None,jsonSongCompare3,""],
    ["",header("wrong"),401,None,"",""],
    ["",header("empty"),401,None,"",""],
    ["",header("nonexist"),400,"","",""],
    ["/a",header("default",auth_keyM),400,None,"",""],
    ["/L$D",header("default",auth_keyM),400,None,"",""],
]

clear(100)

#test get
checkSimpleRequests(gets, "GET", url_songs)

{'Authorization': '29eEJR8VvxyjuTVm', 'Content-Type': 'application/json'}
1 Send request: GET http://localhost:8080/songs/1 {'Authorization': 'zgFiqHXUyFhbStb0', None
{'transfer-encoding': 'chunked', 'Content-Type': 'application/json', 'Date': 'Thu, 30 Sep 2021 17:24:31 GMT'}
 -Recieved: 200 Body: {"id":1,"title":"We Built This City","artist":"Starship","label":"Grunt/RCA","released":1985}
 -Expected: 200 Body: {"id":1,"title":"We Built This City","artist":"Starship","label":"Grunt/RCA","released":1985}
2 Send request: GET http://localhost:8080/songs/1 {'Authorization': '29eEJR8VvxyjuTVmd' None
{'Date': 'Thu, 30 Sep 2021 17:24:31 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
3 Send request: GET http://localhost:8080/songs/1 {'Authorization': '', 'Content-Type': None
{'Date': 'Thu, 30 Sep 2021 17:24:31 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
4 Send request: GET http://localhost:8080/songs/1 {'Content-Type': 'application/jso

## Songs Delete 

In [8]:
# add songs to delete
response = requests.request("POST", url_songs, headers=header("default",auth_keyM), data=jsonSong)
print("Recieved Statuscode: " + str(response.status_code) + " Expected is 201")
assert(201 == response.status_code)
id1 = "/" + str(json.loads(response.text)["id"])

response = requests.request("POST", url_songs, headers=header("default",auth_keyM), data=jsonSong)
print("Recieved Statuscode: " + str(response.status_code) + " Expected is 201")
assert(201 == response.status_code)
id2 = "/" + str(json.loads(response.text)["id"])

deletes = [
    # id , headers, status_code to send with DELETE request
    [id1, header("wrong"), 401, None, "", ""],
    [id1, header("empty"), 401, None, "", ""],
    [id1, header("nonexist"), 400, None, "", ""],
    [id1, header("default",auth_keyM), 204, None, "", ""],
    [id2, header("default",auth_keyM), 204, None, "", ""],
    ["/500", header("default",auth_keyM), 404, None, "", ""],
    ["", header("default",auth_keyM), 400, None, "", ""],
    ["/a", header("default",auth_keyM), 400, None, "", ""],
    ["/$", header("default",auth_keyM), 400, None, "", ""],
    [id1, header("default",auth_keyM), 404, None, "", ""],
]

# test delete
checkSimpleRequests(deletes, "DELETE", url_songs)

Recieved Statuscode: 201 Expected is 201
Recieved Statuscode: 201 Expected is 201
1 Send request: DELETE http://localhost:8080/songs/15 {'Authorization': '29eEJR8VvxyjuTVmd' None
{'Date': 'Thu, 30 Sep 2021 17:24:31 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
2 Send request: DELETE http://localhost:8080/songs/15 {'Authorization': '', 'Content-Type': None
{'Date': 'Thu, 30 Sep 2021 17:24:31 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
3 Send request: DELETE http://localhost:8080/songs/15 {'Content-Type': 'application/json'} None
{'transfer-encoding': 'chunked', 'Content-Type': 'application/json', 'Date': 'Thu, 30 Sep 2021 17:24:31 GMT'}
 -Recieved: 400 Body: {"timestamp":"2021-09-30T17:24:32.454+00:00","status":400,"error":"Bad Request","path":"/songs/15"}
 -Expected: 400 Body: 
4 Send request: DELETE http://localhost:8080/songs/15 {'Authorization': 'zgFiqHXUyFhbStb0', None
{'Date': 'Thu, 30 Sep 2021 17:24:31 GMT'}
 -Recieved: 

## Songs Put Methods

In [9]:
#add songs to put
response = requests.request("POST", url_songs, headers=header("default",auth_keyM), data=jsonSongWithOnlyTitle)
print("Recieved Statuscode: "+ str(response.status_code) + " Expected is 201")
assert(201 == response.status_code)
id1 = str(json.loads(response.text)["id"])

response = requests.request("POST", url_songs, headers=header("default",auth_keyM), data=jsonSongWithOnlyTitle)
print("Recieved Statuscode: "+ str(response.status_code) + " Expected is 201")
assert(201 == response.status_code)
id2 = str(json.loads(response.text)["id"])

jsonSongPut = json.dumps({
  "id": id1,
  "title": "t1",
  "artist": "a1",
  "label": "l1",
  "released": 1901
}, separators=(',', ':'))

jsonSongPut2 = json.dumps({
  "id": id2,
  "title": "t2",
  "artist": "a2",
  "label": "l2",
  "released": 1902
}, separators=(',', ':'))

jsonSongPutWithOnlyTitle = json.dumps({
  "id": id1,
  "title": "t3"
}, separators=(',', ':'))
jsonSongPutWithEmptyTitle = json.dumps({
  "id": id1,
  "title": "",
  "artist": "a4",
  "label": "l4",
  "released": 1904
}, separators=(',', ':'))
jsonSongPutWithOutTitle = json.dumps({
  "id": id1,
  "artist": "a5",
  "label": "l5",
  "released": 1905
}, separators=(',', ':'))

jsonSongPutWithoutArtist = json.dumps({
  "id": id1,
  "title": "t6",
  "label": "l6",
  "released": 1906
}, separators=(',', ':'))

jsonSongPutWithoutLabel = json.dumps({
  "id": id1,
  "title": "t7",
  "artist": "a7",
  "released": 1907
}, separators=(',', ':'))

jsonSongPutWithoutRelease = json.dumps({
  "id": id1,
  "title": "t8",
  "artist": "a8",
  "label": "1908"
}, separators=(',', ':'))

id1 = "/" + id1
id2 = "/" + id2

puts = [
    #id , headers, status_code to send with PUT request
    [id1,header("default",auth_keyM),204,jsonSongPut,"","'Location': '/songs/"],
    [id1,header("default",auth_keyM),400,jsonSongPutWithOutTitle,"",""],
    [id1,header("default",auth_keyM),400,jsonSongPutWithEmptyTitle,"",""],
    [id1,header("default",auth_keyM),204,jsonSongPutWithOnlyTitle,"","'Location': '/songs/"],
    [id1,header("wrong"),401,jsonSongPut,"",""],
    [id1,header("empty"),401,jsonSongPut,"",""],
    [id1,header("nonexist"),400,jsonSongPut,"",""],
    [id1,header("wrong"),401,jsonSongPutWithOutTitle,"",""],
    [id1,header("nonexist"),400,jsonSongPutWithOutTitle,"",""],
    [id2,header("default",auth_keyM),400,jsonSongPut,"",""],
    [id1,header("default",auth_keyM),400,jsonSongPut2,"",""],
    [id1,header("default",auth_keyM),204,jsonSongPutWithoutArtist,"","'Location': '/songs/"],
    [id1,header("default",auth_keyM),204,jsonSongPutWithoutLabel,"","'Location': '/songs/"],
    [id1,header("default",auth_keyM),204,jsonSongPutWithoutRelease,"","'Location': '/songs/"],
    ["/a",header("default",auth_keyM),400,jsonSongPut,"",""]
    ]

#test put
checkSimpleRequests(puts, "PUT",url_songs)

clear()

Recieved Statuscode: 201 Expected is 201
Recieved Statuscode: 201 Expected is 201
1 Send request: PUT http://localhost:8080/songs/17 {'Authorization': 'zgFiqHXUyFhbStb0', {"id":"17","title":"t1","artist":"a1","l
{'Location': '/songs/17', 'Date': 'Thu, 30 Sep 2021 17:24:31 GMT'}
 -Recieved: 204 Body: 
 -Expected: 204 Body: 
{"id":17,"title":"t1","artist":"a1","label":"l1","released":1901}
2 Send request: PUT http://localhost:8080/songs/17 {'Authorization': 'zgFiqHXUyFhbStb0', {"id":"17","artist":"a5","label":"l5","r
{'Date': 'Thu, 30 Sep 2021 17:24:31 GMT', 'content-length': '0'}
 -Recieved: 400 Body: 
 -Expected: 400 Body: 
3 Send request: PUT http://localhost:8080/songs/17 {'Authorization': 'zgFiqHXUyFhbStb0', {"id":"17","title":"","artist":"a4","lab
{'Date': 'Thu, 30 Sep 2021 17:24:31 GMT', 'content-length': '0'}
 -Recieved: 400 Body: 
 -Expected: 400 Body: 
4 Send request: PUT http://localhost:8080/songs/17 {'Authorization': 'zgFiqHXUyFhbStb0', {"id":"17","title":"t3"}
{'Location': 

## Songs Post Methods

In [10]:
posts = [
    #id , headers, status_code to send with post request
    ["",header("default",auth_keyM),201,jsonSong,jsonSong,"'Location': '/songs/"],
    ["",header("default",auth_keyM),201,jsonSongWithOnlyTitle,jsonSongWithOnlyTitle,"'Location': '/songs/"],
    ["",header("default",auth_keyM),400,jsonSongWithOutTitle,"",""],
    ["",header("default",auth_keyM),400,jsonSongWithEmptyTitle,"",""],
    ["",header("wrong"),401,jsonSong,"",""],
    ["",header("empty"),401,jsonSong,"",""],
    ["",header("nonexist"),400,jsonSong,"",""],
    ["",header("wrong"),401,jsonSongWithOutTitle,"",""],
    ["",header("nonexist"),400,jsonSongWithOutTitle,"",""],
    ["/a",header("default",auth_keyM),405,jsonSong,"",""],
    ]


#test post
checkSimpleRequests(posts, "POST",url_songs)

clear()

1 Send request: POST http://localhost:8080/songs {'Authorization': 'zgFiqHXUyFhbStb0', {"title":"Come To Me","artist":"AETNA","
{'transfer-encoding': 'chunked', 'Location': '/songs/19', 'Content-Type': 'application/json', 'Date': 'Thu, 30 Sep 2021 17:24:32 GMT'}
 -Recieved: 201 Body: {"id":19,"title":"Come To Me","artist":"AETNA","label":"unknown","released":2020}
 -Expected: 201 Body: {"title":"Come To Me","artist":"AETNA","label":"unknown","released":2020}
2 Send request: POST http://localhost:8080/songs {'Authorization': 'zgFiqHXUyFhbStb0', {"title":"Come To Me Only Title"}
{'transfer-encoding': 'chunked', 'Location': '/songs/20', 'Content-Type': 'application/json', 'Date': 'Thu, 30 Sep 2021 17:24:32 GMT'}
 -Recieved: 201 Body: {"id":20,"title":"Come To Me Only Title","artist":null,"label":null,"released":null}
 -Expected: 201 Body: {"title":"Come To Me Only Title"}
3 Send request: POST http://localhost:8080/songs {'Authorization': 'zgFiqHXUyFhbStb0', {"artist":"AETNA Without Title"

# Lyrics


### Setup

In [12]:
jsonSongUnknow = json.dumps({
  "title": "Come To Me",
  "artist": "AETNA",
  "label": "unknown",
  "released": 2020
}, separators=(',', ':'))

# add unknown song to for lyrics
response = requests.request("POST", url_songs, headers=header("default",auth_keyM), data=jsonSongUnknow)
print("Recieved Statuscode: " + str(response.status_code) + " Expected is 201")
assert(201 == response.status_code)
id1 = "/" + str(json.loads(response.text)["id"])

headerLyrics = {
    'Artist': 'Michael Jackson',
    'Song': 'Beat it',
    'Authorization': auth_keyM,
    'Content-Type': 'application/json'
}

headerLyricsUnknown = {
    'Artist': 'AETNA',
    'Song': 'Come To Me',
    'Authorization': auth_keyM,
    'Content-Type': 'application/json'
}


Recieved Statuscode: 201 Expected is 201


## Lyrics Get Method

In [13]:

#check lyrics service with known song
print(" Send request: " + "GET", url_lyrics, str(headerLyrics)[:37])
response1 = requests.request(
    "GET", url_lyrics, headers=headerLyrics)
print(str(response1.headers))
print(" -Recieved: " + str(response1.status_code) +
        " Body: "+str(response1.text)[:72])
print(" -Expected: " + str(200) +
        " Body: [Michael Jackson Lyrics]")
assert(200 == response1.status_code), "Status Code Different"

# check lyrics service with unknown song
print(" Send request: " + "GET", url_lyrics, str(headerLyricsUnknown)[:37])
response2 = requests.request(
    "GET", url_lyrics, headers=headerLyricsUnknown)
print(str(response2.headers))
print(" -Recieved: " + str(response2.status_code) +
        " Body: "+str(response2.text))
print(" -Expected: " + str(404) +
        " Body: No results by: http://api.chartlyrics.com/apiv1.asmx/SearchLyricDirect?artist=AETNA&Song=Come+To+Me")
assert(404 == response2.status_code), "Status Code Different"

#check song service using lyric service
print(" Send request: " + "GET", url_songs + "/1/lyrics", header("default",auth_keyM))
response3 = requests.request(
    "GET", url_songs + "/1/lyrics", headers=header("default",auth_keyM))
print(str(response3.headers))
print(" -Recieved: " + str(response3.status_code) +
        " Body: "+str(response3.text)[:72])
print(" -Expected: " + str(200) +
        " Body: [Starship Lyrics]")
assert(200 == response3.status_code), "Status Code Different"


getLyrics = [
    # send request
    #id , headers, status_code, payload, response(expected)
    ["/1/lyrics", header("wrong"), 401, None, "", ""],
    ["/1/lyrics", header("empty"), 401, None, "", ""],
    ["/1/lyrics", header("nonexist"), 400, None, "", ""],

    [id1+"/lyrics", header("default",auth_keyM), 404, None, '', ""],
    [id1+"/lyrics", header("wrong"), 401, None, "", ""],
    [id1+"/lyrics", header("empty"), 401, None, "", ""],
    [id1+"/lyrics", header("nonexist"), 400, None, "", ""],
    
    ["/500/lyrics", header("default",auth_keyM), 404, None, '', ""],
    ["/500/lyrics", header("wrong"), 401, None, "", ""],
    ["/500/lyrics", header("empty"), 401, None, "", ""],
    ["/500/lyrics", header("nonexist"), 400, None, "", ""]
]


# test get
checkSimpleRequests(getLyrics, "GET", url_songs)
clear()


 Send request: GET http://localhost:8080/lyrics {'Artist': 'Michael Jackson', 'Song':
{'transfer-encoding': 'chunked', 'Content-Type': 'application/json', 'Date': 'Thu, 30 Sep 2021 17:33:06 GMT'}
 -Recieved: 200 Body: {"lyricId":1094,"lyric":"They told him don't you ever come around here\n
 -Expected: 200 Body: [Michael Jackson Lyrics]
 Send request: GET http://localhost:8080/lyrics {'Artist': 'AETNA', 'Song': 'Come To 
{'Content-Type': 'text/plain', 'Date': 'Thu, 30 Sep 2021 17:33:06 GMT', 'content-length': '99'}
 -Recieved: 404 Body: No results by: http://api.chartlyrics.com/apiv1.asmx/SearchLyricDirect?artist=AETNA&Song=Come+To+Me
 -Expected: 404 Body: No results by: http://api.chartlyrics.com/apiv1.asmx/SearchLyricDirect?artist=AETNA&Song=Come+To+Me
 Send request: GET http://localhost:8080/songs/1/lyrics {'Authorization': 'zgFiqHXUyFhbStb0', 'Content-Type': 'application/json'}
{'transfer-encoding': 'chunked', 'Content-Type': 'application/json', 'Date': 'Thu, 30 Sep 2021 17:33:06 GM

# SongList


## Songlist Get Methods

In [ ]:
getLists = [
    # send request
    #id , headers, status_code, payload, response(expected)
    ["/1", header("default",auth_keyM), 200, None, '{"id":1,"ownerId":"mmuster","name":"mmusterPrivate","songList":[{"id":1,"title":"We Built This City","artist":"Starship","label":"Grunt/RCA","released":1985},{"id":2,"title":"Sussudio","artist":"Phil Collins","label":"Virgin","released":1985}],"isPrivate":true}', ""],
    ["/1", header("wrong"), 401, None, "", ""],
    ["/1", header("empty"), 401, None, "", ""],
    ["/1", header("nonexist"), 400, None, "", ""],

    ["", header("default",auth_keyM), 200, None, '[{"id":1,"ownerId":"mmuster","name":"mmusterPrivate","songList":[{"id":1,"title":"We Built This City","artist":"Starship","label":"Grunt/RCA","released":1985},{"id":2,"title":"Sussudio","artist":"Phil Collins","label":"Virgin","released":1985}],"isPrivate":true},{"id":2,"ownerId":"mmuster","name":"mmusterPublic","songList":[{"id":1,"title":"We Built This City","artist":"Starship","label":"Grunt/RCA","released":1985},{"id":2,"title":"Sussudio","artist":"Phil Collins","label":"Virgin","released":1985}],"isPrivate":false},{"id":4,"ownerId":"eschuler","name":"eschulerPublic","songList":[{"id":1,"title":"We Built This City","artist":"Starship","label":"Grunt/RCA","released":1985},{"id":2,"title":"Sussudio","artist":"Phil Collins","label":"Virgin","released":1985}],"isPrivate":false}]', ""],
    ["", header("wrong"), 401, None, "", ""],
    ["", header("empty"), 401, None, "", ""],
    ["", header("nonexist"), 400, "", "", ""],

    ["/3", header("default",auth_keyM), 404, None, "", ""],
    ["/3", header("wrong"), 401, None, "", ""],
    ["/3", header("empty"), 401, None, "", ""],
    ["/3", header("nonexist"), 400, None, "", ""],

    ["/a", header("default",auth_keyM), 400, None, "", ""],
    ["/L$D", header("default",auth_keyM), 400, None, "", ""],
    ["/500", header("default",auth_keyM), 404, None, "", ""],

    ["/4", header("default",auth_keyM), 200, None, '{"id":4,"ownerId":"eschuler","name":"eschulerPublic","songList":[{"id":1,"title":"We Built This City","artist":"Starship","label":"Grunt/RCA","released":1985},{"id":2,"title":"Sussudio","artist":"Phil Collins","label":"Virgin","released":1985}],"isPrivate":false}', ""],
    ["/4", header("wrong"), 401, None, "", ""],
    ["/4", header("empty"), 401, None, "", ""],
    ["/4", header("nonexist"), 400, None, "", ""],

    ["/2", header("default",auth_keyM), 200, None, '{"id":2,"ownerId":"mmuster","name":"mmusterPublic","songList":[{"id":1,"title":"We Built This City","artist":"Starship","label":"Grunt/RCA","released":1985},{"id":2,"title":"Sussudio","artist":"Phil Collins","label":"Virgin","released":1985}],"isPrivate":false}', ""],
]

# clear songlists without Id1 and Id2
listsToKeep = 4  # will keep these lists ids, 4 means id1,2,3,4 will be not deleted
for i in range(100):
    requests.request("DELETE", url_songLists + "/" +
                     str(i+listsToKeep+1), headers=header())

# test get
checkSimpleRequests(getLists, "GET", url_songLists)


1 Send request: GET http://localhost:8080/songLists/1 {'Authorization': 'WOxyq5LXyItnLLZp', None
{'transfer-encoding': 'chunked', 'Content-Type': 'application/json', 'Date': 'Thu, 30 Sep 2021 17:11:44 GMT'}
 -Recieved: 200 Body: {"id":1,"ownerId":"mmuster","name":"mmusterPrivate","songList":[{"id":1,"title":"We Built This City","artist":"Starship","label":"Grunt/RCA","released":1985},{"id":2,"title":"Sussudio","artist":"Phil Collins","label":"Virgin","released":1985}],"isPrivate":true}
 -Expected: 200 Body: {"id":1,"ownerId":"mmuster","name":"mmusterPrivate","songList":[{"id":1,"title":"We Built This City","artist":"Starship","label":"Grunt/RCA","released":1985},{"id":2,"title":"Sussudio","artist":"Phil Collins","label":"Virgin","released":1985}],"isPrivate":true}
2 Send request: GET http://localhost:8080/songLists/1 {'Authorization': 'uGE7U5vqhRlcHQXYd' None
{'Date': 'Thu, 30 Sep 2021 17:11:44 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
3 Send request: G

AssertionError: Response Body Different

## Songlist Delete Methods

In [14]:
# add list and songs for list to delete
# add songs to delete
response = requests.request("POST", url_songs, headers=header("default",auth_keyM
), data='{"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}')
print("Recieved Statuscode: " + str(response.status_code) + " Expected is 201")
assert(201 == response.status_code)
songId1 = str(json.loads(response.text)["id"])
response = requests.request("POST", url_songs, headers=header("default",auth_keyM
), data='{"title":"PlaylistSong2","artist":"at2","label":"unknown2","released":2022}')
print("Recieved Statuscode: " + str(response.status_code) + " Expected is 201")
assert(201 == response.status_code)
songId2 = str(json.loads(response.text)["id"])

# add list to delete
response = requests.request("POST", url_songLists, headers=header("default",auth_keyM
), data='{"name":"newPlaylistToDelete","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021},{"id":'+songId2+',"title":"PlaylistSong2","artist":"at2","label":"unknown2","released":2022}],"isPrivate":true}')
print("Recieved Statuscode: " + str(response.status_code) + " Expected is 201")
assert(201 == response.status_code)
id = "/" + str(json.loads(response.text)["id"])


deleteLists = [
    # id , headers, status_code to send with DELETE request
    [id, header("wrong"), 401, None, "", ""],
    [id, header("empty"), 401, None, "", ""],
    [id, header("nonexist"), 400, None, "", ""],
    [id, header("default",auth_keyM), 204, None, "", ""],
    ["/3", header("default",auth_keyM), 401, None, "", ""],
    ["/4", header("default",auth_keyM), 401, None, "", ""],
    [id, header("default",auth_keyM), 404, None, "", ""],
    ["/a", header("default",auth_keyM), 400, None, "", ""],
    ["/500", header("default",auth_keyM), 404, None, "", ""],
    ["", header("default",auth_keyM), 400, None, "", ""],
    ["/$", header("default",auth_keyM), 400, None, "", ""],
]


# test delete
checkSimpleRequests(deleteLists, "DELETE", url_songLists)


Recieved Statuscode: 201 Expected is 201
Recieved Statuscode: 201 Expected is 201
Recieved Statuscode: 201 Expected is 201
1 Send request: DELETE http://localhost:8080/songLists/11 {'Authorization': '29eEJR8VvxyjuTVmd' None
{'Date': 'Thu, 30 Sep 2021 17:33:18 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
2 Send request: DELETE http://localhost:8080/songLists/11 {'Authorization': '', 'Content-Type': None
{'Date': 'Thu, 30 Sep 2021 17:33:18 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
3 Send request: DELETE http://localhost:8080/songLists/11 {'Content-Type': 'application/json'} None
{'transfer-encoding': 'chunked', 'Content-Type': 'application/json', 'Date': 'Thu, 30 Sep 2021 17:33:18 GMT'}
 -Recieved: 400 Body: {"timestamp":"2021-09-30T17:33:18.952+00:00","status":400,"error":"Bad Request","path":"/songLists/11"}
 -Expected: 400 Body: 
4 Send request: DELETE http://localhost:8080/songLists/11 {'Authorization': 'zgFiqHXUyFhbStb0'

## Songlist Put Methods

In [15]:

# add list and songs for list
# add songs to
response = requests.request("POST", url_songs, headers=header("default",auth_keyM
), data='{"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}')
print("Recieved Statuscode: " + str(response.status_code) + " Expected is 201")
assert(201 == response.status_code)
songId1 = str(json.loads(response.text)["id"])

response = requests.request("POST", url_songs, headers=header("default",auth_keyM
), data='{"title":"PlaylistSong2","artist":"at2","label":"unknown2","released":2022}')
print("Recieved Statuscode: " + str(response.status_code) + " Expected is 201")
assert(201 == response.status_code)
songId2 = str(json.loads(response.text)["id"])

print(songId1)
print(songId2)

jsonSonglistPost = '{"name":"newPlaylistToPut","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021},{"id":'+songId2+',"title":"PlaylistSong2","artist":"at2","label":"unknown2","released":2022}],"isPrivate":true}'

# add list to put
response = requests.request("POST", url_songLists,
                            headers=header("default",auth_keyM), data=jsonSonglistPost)
print(response.text)
print(header("default",auth_keyM))
print("Recieved Statuscode: " + str(response.status_code) + " Expected is 201")
assert(201 == response.status_code)
id = str(json.loads(response.text)["id"])

jsonSonglistPut = '{"id":'+id+',"ownerId":"mmuster","name":"newPlaylistToPut","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":true}'
jsonSonglistPutNot = '{"id":'+id+',"ownerId":"mmuster","name":"newPlaylistToPutNOT","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":true}'
jsonSonglistPut3 = '{"id":3,"ownerId":"mmuster","name":"newPlaylistToPutNOT","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":true}'
jsonSonglistPut4 = '{"id":4,"ownerId":"mmuster","name":"newPlaylistToPutNOT","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":true}'
jsonSonglistPutIdWrong = '{"id":"500,"ownerId":"mmuster","name":"newPlaylistToPut","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":true}'
jsonSonglistPut_changed = '{"id":'+id+',"ownerId":"mmuster","name":"newPlaylistToPut_changed","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":false}'
jsonSongListPutWithOutName = '{"id":'+id+',"ownerId":"mmuster","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":false}'
jsonSongListPutWithEmptyName = '{"id":'+id+',"ownerId":"mmuster","name":"","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":false}'
jsonSongListPutWithOnlyName = '{"id":'+id+',"ownerId":"mmuster","name":"WithOnlyName"}'

jsonSonglistRec = '{"id":'+id+',"ownerId":"mmuster","name":"newPlaylistToPut","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":true}'
jsonSonglistRec_changed = '{"id":'+id+',"ownerId":"mmuster","name":"newPlaylistToPut_changed","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":false}'
jsonSongListRecWithOnlyName = '{"id":'+id+',"ownerId":"mmuster","name":"WithOnlyName","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":false}'


putLists = [
    # id , headers, status_code to send with PUT request
    ["/"+id, header("wrong"), 401, jsonSonglistPut_changed, "", ""],
    ["/"+id, header("empty"), 401, jsonSonglistPut_changed, "", ""],
    ["/"+id, header("nonexist"), 400, jsonSonglistPut_changed, "", ""],
    ["/"+id, header("default",auth_keyM), 200, jsonSonglistPut_changed, jsonSonglistRec_changed, ""],

    ["/3", header("default",auth_keyM), 401, jsonSonglistPut3, "", ""],
    ["/4", header("default",auth_keyM), 401, jsonSonglistPut4, "", ""],

    ["/"+id, header("default",auth_keyM), 200, jsonSongListPutWithOutName, jsonSonglistRec_changed, ""],
    ["/"+id, header("default",auth_keyM), 200, jsonSongListPutWithEmptyName, jsonSonglistRec_changed, ""],
    ["/"+id, header("default",auth_keyM), 200, jsonSongListPutWithOnlyName,
     jsonSongListRecWithOnlyName, ""],

    ["/"+id, header("default",auth_keyM), 400, jsonSonglistPutIdWrong, "", ""],
    ["/2", header("default",auth_keyM), 400, jsonSonglistPutNot, "", ""],
    ["/500", header("default",auth_keyM), 400, jsonSonglistPutNot, "", ""],
    ["/a", header("default",auth_keyM), 400, jsonSonglistPutNot, "", ""],
]

# test put
checkSimpleRequests(putLists, "PUT", url_songLists)

clear()

Recieved Statuscode: 201 Expected is 201
Recieved Statuscode: 201 Expected is 201
24
25
{"id":12,"ownerId":"mmuster","name":"newPlaylistToPut","songList":[{"id":24,"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021},{"id":25,"title":"PlaylistSong2","artist":"at2","label":"unknown2","released":2022}],"isPrivate":true}
{'Authorization': 'zgFiqHXUyFhbStb0', 'Content-Type': 'application/json'}
Recieved Statuscode: 201 Expected is 201
1 Send request: PUT http://localhost:8080/songLists/12 {'Authorization': '29eEJR8VvxyjuTVmd' {"id":12,"ownerId":"mmuster","name":"new
{'Date': 'Thu, 30 Sep 2021 17:33:21 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
2 Send request: PUT http://localhost:8080/songLists/12 {'Authorization': '', 'Content-Type': {"id":12,"ownerId":"mmuster","name":"new
{'Date': 'Thu, 30 Sep 2021 17:33:21 GMT', 'content-length': '0'}
 -Recieved: 401 Body: 
 -Expected: 401 Body: 
3 Send request: PUT http://localhost:8080/songLists/1

## Songlist Post Methods

In [ ]:
# add lists and songs for list
# add songs
response = requests.request("POST", url_songs, headers=header("default",auth_keyM
), data='{"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}')
print("Recieved Statuscode: " + str(response.status_code) + " Expected is 201")
assert(201 == response.status_code)
songId1 = str(json.loads(response.text)["id"])

response = requests.request("POST", url_songs, headers=header("default",auth_keyM
), data='{"title":"PlaylistSong2","artist":"at2","label":"unknown2","released":2022}')
print("Recieved Statuscode: " + str(response.status_code) + " Expected is 201")
assert(201 == response.status_code)
songId2 = str(json.loads(response.text)["id"])

print(songId1)
print(songId2)

# add lists
jsonSonglistPost = '{"name":"newPlaylistToPost","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":true}'
jsonSonglistPostPub = '{"name":"newPlaylistToPostPublic","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}]}'
jsonSonglistPostNot = '{"name":"newPlaylistToPostNOT","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":true}'
jsonSonglistPostIdWrong = '{"name":"newPlaylistToPost","songList":[{"id":500,"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":true}'
jsonSongListPostWithOutName = '{"songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":false}'
jsonSongListPostWithEmptyName = '{"name":"","songList":[{"id":'+songId1+',"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":false}'
jsonSongListPostWithOnlyName = '{"name":"WithOnlyName"}'

postLists = [
    #id , headers, status_code to send with PUT request
    ["",header("default",auth_keyM),201,jsonSonglistPost,jsonSonglistPost,"'Location': '/songLists/"],
    ["",header("default",auth_keyM),201,jsonSonglistPostPub,jsonSonglistPostPub,"'Location': '/songLists/"],
    ["",header("default",auth_keyM),201,jsonSongListPostWithOnlyName,jsonSongListPutWithOnlyName,"'Location': '/songLists/"],
    ["",header("default",auth_keyM),400,jsonSongListPostWithOutName,"",""],
    ["",header("default",auth_keyM),400,jsonSongListPostWithEmptyName,"",""],
    ["",header("default",auth_keyM),400,jsonSonglistPostIdWrong,"",""],
    ["",header("wrong"),401,jsonSonglistPostNot,"",""],
    ["",header("empty"),401,jsonSonglistPostNot,"",""],
    ["",header("nonexist"),400,jsonSonglistPostNot,"",""],
    ["",header("wrong"),401,jsonSongListPostWithOutName,"",""],
    ["",header("nonexist"),400,jsonSongListPostWithOutName,"",""],
    ["/a",header("default",auth_keyM),405,jsonSonglistPostNot,"",""],
    ]

# test post
checkSimpleRequests(postLists, "POST", url_songLists)

clear()

Recieved Statuscode: 201 Expected is 201
Recieved Statuscode: 201 Expected is 201
13
14
1 Send request: POST http://localhost:8080/songLists {'Authorization': 'WOxyq5LXyItnLLZp', {"name":"newPlaylistToPost","songList":[
{'transfer-encoding': 'chunked', 'Location': '/songLists/7', 'Content-Type': 'application/json', 'Date': 'Thu, 30 Sep 2021 17:12:00 GMT'}
 -Recieved: 201 Body: {"id":7,"ownerId":"mmuster","name":"newPlaylistToPost","songList":[{"id":13,"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":true}
 -Expected: 201 Body: {"name":"newPlaylistToPost","songList":[{"id":13,"title":"PlaylistSong1","artist":"at1","label":"unknown1","released":2021}],"isPrivate":true}
2 Send request: POST http://localhost:8080/songLists {'Authorization': 'WOxyq5LXyItnLLZp', {"name":"newPlaylistToPostPublic","songL
{'transfer-encoding': 'chunked', 'Location': '/songLists/8', 'Content-Type': 'application/json', 'Date': 'Thu, 30 Sep 2021 17:12:00 GMT'}
 -Recieved: 201